# Demonstrate FFT padding and windowing


Since the FFT is much faster for powers of 2, we often have to "pad" our data (fill it with a constant value) to satisfy that. This is also used in other applications like image processing. 

However, this is mathematically equivalent to multiplying a function by a step function, which induces characteristic "ringing" effects. "Windowing" the input transform can reduce this ringing (at the expense of loss of information on the boundaries of the input waveform). 

Thought question: If you window, transform, transform back, and reverse the window, what should occur? You'll explore this in your HW assignments. 

In [ ]:
import matplotlib.pyplot as plt

from fft import fft, fft_power
import numpy as np
import math
from functools import partial

In [ ]:
# Make the plots a bit bigger to see
# NOTE: Must be done in a separate cell
plt.rcParams['figure.dpi'] = 90

## Simple Sine Wave

In this example, since it is a simple sine wave, it goes on forever. So, in our case, to demonstrate what is happening we will clip off a certain number of entries (see the "clip" variable). 

In [ ]:
N = 1024 # number of samples
k = 10.0 # frequency = k / N
m = 0.   # linear slope, if desired

# For demonstrations : 
clip = 40       # "clip" so we can pad with zeros

def apply_clip(iy, Nclip):
    ret = iy.copy()
    ret[-1*Nclip:] = 0
    return ret

# Generate the input data
x = np.arange(N) 
y = np.sin( -2. * np.pi * k / N * x ) 
y_clip = apply_clip(y, 100)

# Plot input data
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(x, y, label="Unmodified")
ax.plot(x, y_clip, label="Clipped")
ax.set_xlim(0., 1500.)
ax.set_ylim(-1.5, 1.5)
ax.grid()
ax.legend()
'''
if window:
    y = np.sin( -2. * np.pi * k / N * x ) * (0.5 - 0.5 * np.cos(2*math.pi*x/float(N-1))) + m*x
else:
    y = np.sin( -2. * np.pi * k / N * x ) 
'''

Next, we show what the clipping does to the Fourier transform of our function. Remember, the FT of the sine wave is just a "delta function" at the input frequency. The follow cell shows that the "clipping" introduces extra Fourier components at undesired frequencies.

In [ ]:
# Let's just use numpy's FFT implementation here
Y = fft(y)
Y_clip = fft(y_clip)

Ypower = fft_power(Y)
x_power = np.arange(len(Ypower))
Ypower_clip = fft_power(Y_clip)

fig, axs = plt.subplots(3, 1, figsize=(12, 12))
axs[0].plot(x, y, label="Unmodified", color="blue")
axs[0].plot(x, y_clip, label="Clipped", color="orange")
axs[0].set_xlabel("x")
axs[0].set_ylabel("sin(x)")
axs[0].legend()
axs[0].set_xlim(0., 1500.)

axs[1].scatter(x_power, Ypower, label="Unmodified", marker="o", facecolor="blue")
axs[1].scatter(x_power, Ypower_clip, label="Clipped", marker="o", facecolor="none", edgecolor="orange")
axs[1].set_xlabel("Spectral index")
axs[1].set_ylabel("Power")
axs[1].set_xlim(0., 20.)
axs[1].set_ylim(1.e-32, 1.e9)
axs[1].set_yscale("log")

axs[2].scatter(x_power, Ypower, label="Unmodified", marker="o", facecolor="blue")
axs[2].scatter(x_power, Ypower_clip, label="Clipped", marker="o", facecolor="none", edgecolor="orange")
axs[2].set_xlabel("Spectral index")
axs[2].set_ylabel("Power (zoomed)")
axs[2].set_xlim(0., 550.)
axs[2].set_ylim(1.e-7, 1.e5)
axs[2].set_yscale("log")



# Windowing
Clipping (or conversely, zero-padding) has the unwanted effect of introducing extra Fourier components. Essentially, the clipped function is the product of our original function and a "box" function, so the Fourier transform is the convolution of the FT of our function and the box function. This "smears out" the intended FT. 

We can clean up the unwanted components by applying a window function. Essentially, we are changing the box function (whose FT is not very nice) to better function whose FT is less "spread out". We choose a "raised cosine" function here.

In [ ]:
def apply_window(ix, iy):
    return iy * (0.5 - 0.5 * np.cos(2 * math.pi * ix / float(N-1))) + m * ix

y_window = apply_window(x, y)
y_window_clip = apply_clip(y_window, 50)

# Plot input data
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(x, y, label="Unmodified")
ax.plot(x, y_clip, label="Clipped")
ax.plot(x, y_window, label="Windowed")
ax.plot(x, y_window_clip, label="Clipped*windowed")
ax.set_xlim(0., 1500.)
ax.set_ylim(-1.5, 1.5)
ax.grid()
ax.legend()


In [ ]:
# Let's just use numpy's FFT implementation here
Y_window_clip = fft(y_window_clip)
Ypower_window_clip = fft_power(Y_window_clip)

fig, axs = plt.subplots(3, 1, figsize=(12, 12))
axs[0].plot(x, y, label="Unmodified", color="blue")
axs[0].plot(x, y_clip, label="Clipped", color="orange")
axs[0].plot(x, y_window_clip, label="Clipped*windowed", color="green")
axs[0].set_xlabel("x")
axs[0].set_ylabel("sin(x)")
axs[0].legend()
axs[0].set_xlim(0., 1800.)

axs[1].scatter(x_power, Ypower, label="Unmodified", marker="o", facecolor="blue")
axs[1].scatter(x_power, Ypower_clip, label="Clipped", marker="o", facecolor="none", edgecolor="orange")
axs[1].scatter(x_power, Ypower_window_clip, label="Clipped*windowed", marker="o", facecolor="none", edgecolor="green")
axs[1].set_xlabel("Spectral index")
axs[1].set_ylabel("Power")
axs[1].set_xlim(0., 20.)
axs[1].set_ylim(1.e-32, 1.e9)
axs[1].set_yscale("log")

axs[2].scatter(x_power, Ypower, label="Unmodified", marker="o", facecolor="blue")
axs[2].scatter(x_power, Ypower_clip, label="Clipped", marker="o", facecolor="none", edgecolor="orange")
axs[2].scatter(x_power, Ypower_window_clip, label="Clipped*windowed", marker="o", facecolor="none", edgecolor="green")
axs[2].set_xlabel("Spectral index")
axs[2].set_ylabel("Power (zoomed)")
axs[2].set_xlim(0., 20.)
axs[2].set_ylim(1.e-7, 1.e9)
axs[2].set_yscale("log")



## CO$_2$ Data

These are real data, but are not a power of 2. In this case, we need to pad the input distribution. You can check the effect of windowing with the "window" method. 

In [ ]:
from read_co2 import read_co2
import math

# Apply window?
window = True

# Read like previous example with CO2 data
x,y = read_co2('co2_mm_mlo.txt')
y_valid = y >= 0.
y = y[y_valid]

# instead of truncating, pad with values
N = len(y)
log2N_ceil = math.ceil(math.log(N, 2))
#log2N = math.log(N, 2)
#next_pow_of_2 = int(log2N) + 1
if N != 2**log2N_ceil:
    # Create new array from inputs + pad values
    print(f"Padding input array of length {N} to length {2**log2N_ceil}")
    pad_value = 300
    y_new = np.ones(2**log2N_ceil) * pad_value
    y_new[:len(y)] = y
    y = y_new
    
    #ypads = np.full( 2**( next_pow_of_2) - N, 300, dtype=np.double)
    #y = np.concatenate( (y, ypads) )
    # CAREFUL: When you pad, the x axis becomes somewhat "meaningless" for the padded values, 
    # so typically it is best to just consider it an index
    x = np.arange(len(y))
    N = len(y)
    # Apply a window to reduce ringing from the 2^n cutoff
    if window : 
        y = y * (0.5 - 0.5 * np.cos(2*np.pi*x/(N-1)))
                

Y = fft(y)
Y_abs = abs(Y)
powery = fft_power(Y)
powerx = np.arange(powery.size)

f1 = plt.figure(1)
plt.plot( x, y )
plt.xlabel("Index")
plt.ylabel("CO$_2$ Concentration")

f2 = plt.figure(2)
plt.plot( powerx, powery, label="Power" )
plt.plot( x, Y_abs, label="Magnitude" )
plt.xlim([0,N/4])
plt.legend()
plt.yscale('log')
plt.xlabel("Spectral Index")
plt.ylabel("Fourier Component")

plt.show()



In [ ]:
# Compare different windows
windows_to_plot = ["bartlett"]
# Options: ["bartlett", "hanning", "hamming", "blackman", "kaiser"]

wfuncs = {
    "bartlett": np.bartlett, 
    "hanning": np.hanning, 
    "hamming": np.hamming, 
    "blackman": np.blackman, 
    "kaiser": partial(np.kaiser, beta=3)
}

def pad_data(y, pad_value=0):
    N = len(y)
    log2N_ceil = math.ceil(math.log(N, 2))
    #log2N = math.log(N, 2)
    #next_pow_of_2 = int(log2N) + 1
    if N == 2**log2N_ceil:
        # Input is already a power of 2, nothing to do
        return y
    else:
        # Create new array from inputs + pad values
        print(f"Padding input array of length {N} to length {2**log2N_ceil}")
        y_new = np.ones(2**log2N_ceil) * pad_value
        y_new[:len(y)] = y
        return y_new

# y = input data to be windowed
# wname = window name (see https://numpy.org/doc/stable/reference/routines.window.html)
# M = length of good data
def apply_window(y, wname, M):
    window = np.zeros(len(y))
    window[:M] = wfuncs[wname](M)
    return y * window

# Read like previous example with CO2 data
x0, y0 = read_co2('co2_mm_mlo.txt')
y0_valid = y0 >= 0.
y0 = y0[y_valid]
# Instead padding with 300s, subtract the DC offset first
y0 = y0 - y0[0]


# Transformed input data
y = {}
y["padded"] = pad_data(y0)
for wname in wfuncs.keys():
    y[wname] = apply_window(y["padded"], wname, len(y0))

# Fourier transforms and powers
Ypower = {}
Ypower["padded"] = fft_power(np.fft.fft(y["padded"]))
for wname in windows_to_plot:
    Ypower[wname] = fft_power(np.fft.fft(y[wname]))

# Also do DFT on original data for comparison
Ypower["original"] = fft_power(np.fft.fft(y0))

# Plot
fig, axs = plt.subplots(2, 1, figsize=(12, 10))
x = np.arange(len(y["padded"]))
xpower = np.arange(len(Ypower["padded"]))
for wname in ["padded"] + windows_to_plot:
    axs[0].plot(x, y[wname], label=wname)
    axs[1].plot(xpower, Ypower[wname], label=wname)

# Stretch out x-axis for the original FT, to match the padded one
axs[0].plot(np.arange(len(y0)), y0, label="original")
axs[1].plot(np.arange(len(Ypower["original"])) * len(Ypower["padded"]) / len(Ypower["original"]), Ypower["original"], label="original")

axs[0].set_xlabel("Month")
axs[0].set_ylabel(r"$\Delta$CO2 concentration")
axs[0].legend()

axs[1].set_xlabel("Spectral index")
axs[1].set_ylabel("Power")
axs[1].set_yscale("log")